In [1]:
import re
import gc
import pickle
import argparse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import *
from keras import optimizers
from keras import backend as K
from collections import Counter
from keras.utils import np_utils
from gensim.models import word2vec
from sklearn.metrics import f1_score
from keras.regularizers import l1, l2
from keras.engine import InputSpec, Layer
from keras.models import Model, load_model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.layers import Input, Embedding, Dense
from sklearn.preprocessing import StandardScaler
from keras import optimizers, losses, activations
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
from keras.layers.noise import GaussianNoise
from models_def import Attention
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from keras.callbacks import LearningRateScheduler
config = argparse.Namespace()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
der=True
if der:
    train_dir = '../input/new_data/train_set.csv'
    test_dir = '../input/new_data/test_set.csv'
else:
#     train_dir = '../input/new_data/train_remove60.csv'
#     test_dir = '../input/new_data/test_remove60.csv'
    train_dir = '../input/new_data/shuffle_train.csv'
    test_dir = '../input/new_data/test_set.csv'

train = pd.read_csv(train_dir)
test = pd.read_csv(test_dir)
# original_train = pd.read_csv('../input/new_data/train_set.csv')
# shuffle_data = train[original_train.shape[0]:]

In [3]:
df_y_train = (train["class"]-1).astype(int)
test_id = test[["id"]].copy()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
pseudo_label_data = pd.read_csv('../10fold_pro/prob_rnn_baseline4.csv')
name = ["class_prob_%s"%i for i in range(1,20)]
pseudo_y=np.argmax(pseudo_label_data[name].values,axis=1)
# all_y = np.append(df_y_train, preds)

In [5]:
all_data = pd.concat([train, test])
all_data = all_data.reset_index(drop=True)
train_offset = train.shape[0]

In [6]:
column = "word_seg"
config.len_desc = 800000
tknzr_word = Tokenizer(num_words=config.len_desc)
tknzr_word.fit_on_texts(all_data[column].values)

In [7]:
count_thres = 5
low_count_words = [w for w, c in tknzr_word.word_counts.items() if c < count_thres]
# print(len(tknzr_word.texts_to_sequences(all_data[column].values)))
for w in tqdm(low_count_words):
    del tknzr_word.word_index[w]
    del tknzr_word.word_docs[w]
    del tknzr_word.word_counts[w]

100%|██████████| 912182/912182 [00:01<00:00, 731547.37it/s]


In [8]:
tr_word_seq = tknzr_word.texts_to_sequences(train[column].values)
te_word_seq = tknzr_word.texts_to_sequences(test[column].values)
config.maxlen = 1000
tr_word_pad = pad_sequences(tr_word_seq, maxlen=config.maxlen)
te_word_pad = pad_sequences(te_word_seq, maxlen=config.maxlen)

In [9]:
from gensim.models import word2vec
max_features = 800000
vec_len = 600
EMBEDDING = '../feature/word2vec_file/avito600d.w2v'
model = word2vec.Word2Vec.load(EMBEDDING)
word_index = tknzr_word.word_index
nb_words_desc = min(max_features, len(word_index))
embedding_matrix_desc = np.zeros((nb_words_desc+1, vec_len))
for word, i in word_index.items():
    if i >= max_features: continue
    try:
        embedding_vector = model[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix_desc[i] = embedding_vector

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [10]:
Y = df_y_train
pseudo_Y = pseudo_y

In [11]:
from keras.engine import InputSpec, Layer
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [12]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [13]:
from keras.layers import Input, Embedding, Dense
# from keras.layers import GlobalMaxPool1D, GlobalMaxPool2D, SpatialDropout1D, Dropout, BatchNormalization, Lambda
# from keras.layers import concatenate, Flatten, add, dot, PReLU, merge, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Add, Multiply
# from keras.layers import LSTM, Conv1D, GlobalMaxPool2D, Convolution2D, Conv2D, CuDNNGRU, CuDNNLSTM, GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.regularizers import l1, l2
from keras import optimizers, losses, activations
from keras import backend as K
from keras import optimizers
from keras.layers import *
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [14]:
def get_rnn_model():
#     features_input = Input(shape=(features.shape[1],))

    
    inpword = Input(shape=(config.maxlen, ))
    emb_word = Embedding(embedding_matrix_desc.shape[0], 600,  weights = [embedding_matrix_desc],trainable=False)(inpword)

    lDropout_titl = SpatialDropout1D(0.5)(emb_word)
    title_layer = Bidirectional(CuDNNLSTM(128, return_sequences=True))(lDropout_titl)

    title_layer = Bidirectional(CuDNNGRU(128, return_sequences=True))(title_layer)

    max_pool_til = GlobalMaxPooling1D()(title_layer)
    att = AttentionWeightedAverage()(title_layer)
# AttentionWeightedAverage()

    all_views = concatenate([max_pool_til, att], axis=1)
    x = Dropout(0.5)(all_views)

    x = PReLU()(Dense(128)(x))

    x = Dense(19, activation="softmax")(x)
    model = Model(inputs=[inpword], outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    
    return model

In [15]:
one_y_train=np_utils.to_categorical(df_y_train,num_classes=19)
pseudo_y_train=np_utils.to_categorical(pseudo_Y,num_classes=19)

In [16]:
decay_dic = {0:0.001, 1:0.001, 2:0.0009, 3:0.0008, 4:0.0007, 5:0.0006, 6:0.0005, 7:0.0004, 8:0.0003, 9:0.0002, 10:0.0001,
            11:0.00009, 12:0.00008, 13:0.00007, 14:0.00006, 15:0.00005}
def lr_decay(epoch):
    return decay_dic[epoch]
from sklearn.model_selection import KFold
# pretrain=False
# if pretrain == True and len(X_test)<38:
#     X_test.append(test_x)
def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((102277,19)),np.zeros((102277,19))
    LRDecay = LearningRateScheduler(lr_decay)
    for train_index, test_index in kf.split(train):
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x1 = tr_word_pad[train_index]
        hold_out_x1=tr_word_pad[test_index]
        pseudo_number = 1
#         pseudo_word_pad = tr_word_pad[original_train.shape[0]:]
#         pseudo_word_pad_y = one_y_train[original_train.shape[0]:]
        for pseudo_train_index, pseudo_test_index in kf.split(te_word_pad):
            pseudo_train = te_word_pad[pseudo_train_index]
            pseudo_train_y = pseudo_y_train[pseudo_train_index]
            if pseudo_number == now_nfold:
                break
            pseudo_number+=1
        curr_x1 = np.vstack((curr_x1, pseudo_train))
        curr_y, hold_out_y = one_y_train[train_index], one_y_train[test_index]
        curr_y = np.vstack((curr_y, pseudo_train_y))

        config.batch_size = 64
        epochs = 15

        model = get_rnn_model()

        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, save_best_only=True,verbose=1, monitor='val_f1',  mode='max')
        early = EarlyStopping( monitor='val_f1',  mode='max', patience=2,)
        callbacks_list = [checkpoint,early,LRDecay]

        model.fit(curr_x1, curr_y, 
                  batch_size=config.batch_size, epochs=epochs, 
                    validation_data=(hold_out_x1, hold_out_y), 
                  callbacks=callbacks_list)

        
        model.load_weights(file_path)
       
        y_test = model.predict(te_word_pad)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x1)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    try:
        print('all eval',sqrt(mean_squared_error(Y,train_pred)))
    finally:
        return train_pred, test_pred


print('def done')

def done


In [17]:
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)

now is 1 fold
Train on 184098 samples, validate on 10228 samples
Epoch 1/15
184098/184098 [==============================] - 1409s 8ms/step - loss: 0.8742 - f1: 0.7290 - val_loss: 0.8687 - val_f1: 0.7700

Epoch 00001: val_f1 improved from -inf to 0.77001, saving model to weights_base.best.h5
Epoch 2/15
184098/184098 [==============================] - 1425s 8ms/step - loss: 0.6443 - f1: 0.8091 - val_loss: 0.8235 - val_f1: 0.7816

Epoch 00002: val_f1 improved from 0.77001 to 0.78165, saving model to weights_base.best.h5
Epoch 3/15
184098/184098 [==============================] - 1437s 8ms/step - loss: 0.5891 - f1: 0.8260 - val_loss: 0.8031 - val_f1: 0.7909

Epoch 00003: val_f1 improved from 0.78165 to 0.79090, saving model to weights_base.best.h5
Epoch 4/15
184098/184098 [==============================] - 1441s 8ms/step - loss: 0.5537 - f1: 0.8376 - val_loss: 0.7914 - val_f1: 0.7958

Epoch 00004: val_f1 improved from 0.79090 to 0.79577, saving model to weights_base.best.h5
Epoch 5/15
184

184098/184098 [==============================] - 1452s 8ms/step - loss: 0.8764 - f1: 0.7291 - val_loss: 0.8492 - val_f1: 0.7706

Epoch 00001: val_f1 improved from -inf to 0.77063, saving model to weights_base.best.h5
Epoch 2/15
184098/184098 [==============================] - 1457s 8ms/step - loss: 0.6472 - f1: 0.8085 - val_loss: 0.7919 - val_f1: 0.7861

Epoch 00002: val_f1 improved from 0.77063 to 0.78610, saving model to weights_base.best.h5
Epoch 3/15
184098/184098 [==============================] - 1457s 8ms/step - loss: 0.5878 - f1: 0.8270 - val_loss: 0.7812 - val_f1: 0.7899

Epoch 00003: val_f1 improved from 0.78610 to 0.78992, saving model to weights_base.best.h5
Epoch 4/15
184098/184098 [==============================] - 1458s 8ms/step - loss: 0.5551 - f1: 0.8372 - val_loss: 0.7897 - val_f1: 0.7935

Epoch 00004: val_f1 improved from 0.78992 to 0.79346, saving model to weights_base.best.h5
Epoch 5/15
184098/184098 [==============================] - 1463s 8ms/step - loss: 0.5299 

184098/184098 [==============================] - 1424s 8ms/step - loss: 0.4168 - f1: 0.8797 - val_loss: 0.7521 - val_f1: 0.8046

Epoch 00014: val_f1 improved from 0.80450 to 0.80458, saving model to weights_base.best.h5
Epoch 15/15
184098/184098 [==============================] - 1424s 8ms/step - loss: 0.4132 - f1: 0.8818 - val_loss: 0.7538 - val_f1: 0.8057

Epoch 00015: val_f1 improved from 0.80458 to 0.80567, saving model to weights_base.best.h5
now is 7 fold
Train on 184098 samples, validate on 10228 samples
Epoch 1/15
184098/184098 [==============================] - 1419s 8ms/step - loss: 0.8721 - f1: 0.7293 - val_loss: 0.8688 - val_f1: 0.7665

Epoch 00001: val_f1 improved from -inf to 0.76646, saving model to weights_base.best.h5
Epoch 2/15
184098/184098 [==============================] - 1421s 8ms/step - loss: 0.6455 - f1: 0.8088 - val_loss: 0.8262 - val_f1: 0.7830

Epoch 00002: val_f1 improved from 0.76646 to 0.78295, saving model to weights_base.best.h5
Epoch 3/15
184098/184098


Epoch 00012: val_f1 improved from 0.79948 to 0.79952, saving model to weights_base.best.h5
Epoch 13/15
184100/184100 [==============================] - 1447s 8ms/step - loss: 0.4175 - f1: 0.8800 - val_loss: 0.8067 - val_f1: 0.8003

Epoch 00013: val_f1 improved from 0.79952 to 0.80027, saving model to weights_base.best.h5
Epoch 14/15
184100/184100 [==============================] - 1459s 8ms/step - loss: 0.4131 - f1: 0.8807 - val_loss: 0.8063 - val_f1: 0.8008

Epoch 00014: val_f1 improved from 0.80027 to 0.80079, saving model to weights_base.best.h5
Epoch 15/15
184100/184100 [==============================] - 1424s 8ms/step - loss: 0.4105 - f1: 0.8816 - val_loss: 0.8104 - val_f1: 0.8011

Epoch 00015: val_f1 improved from 0.80079 to 0.80114, saving model to weights_base.best.h5
now is 10 fold
Train on 184100 samples, validate on 10227 samples
Epoch 1/15
184100/184100 [==============================] - 1392s 8ms/step - loss: 0.8787 - f1: 0.7272 - val_loss: 0.8733 - val_f1: 0.7669

Epoch 

In [18]:
with open('../feature/rnn_pseudo1.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

In [19]:
val_result = np.argmax(train_pred,axis=1)
print ('score is {}'.format(f1_score(Y, val_result, average='macro')))

score is 0.785668326486411


In [20]:
test_prob=pd.DataFrame(test_pred)
test_prob.columns=["class_prob_%s"%i for i in range(1,test_pred.shape[1]+1)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv('../pro/prob_pseudo_rnn.csv',index=None)

In [21]:
preds=np.argmax(test_pred,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/sub_rnn_pseudo.csv',index=None)

(102277, 1)
(102277, 1)
